In [1]:
print('Hello')

Hello


#### Today i will start working on Myra chatbot which will automate many tasks and give you comfort and smooth experience.

In [3]:
from crewai import Crew,Agent,Task,Process

c:\Users\tjwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_generate_schema.py:777: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [4]:
from keys import GOOGLE_API_KEY,SERPER_API_KEY
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai_tools import SerperDevTool,ScrapeWebsiteTool
import os


c:\Users\tjwin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash',
    temperature=0.5,
    verbose=True,
    google_api_key = GOOGLE_API_KEY,
)



os.environ["SERPER_API_KEY"] = SERPER_API_KEY
serper = SerperDevTool()
scrape = ScrapeWebsiteTool("https://www.makemytrip.com/")

In [6]:
scrape = ScrapeWebsiteTool(config={"headers": {"User-Agent": "BookingAgent/1.0"}})

In [11]:
booking_agent = Agent(
    role = "Booking Specialist",
    goal = "Streamline a flights or trains (which ever will be the best)from {start_trip} to {destination} and hotels in {destination} for  with minimal hassle in {Budget} for {people} people with {day} stay and ",
    backstory = ("""You are skilled in finding the best deals and securing bookings quickly at cheapest."""),
    memory = True,
    allow_deligation = True,
    verbose = True,
    tools = [serper,scrape],
    llm = model)

In [12]:
booking_task = Task(
    description = ("""streamline the flights or trains(which ever will be the best) from {start_trip} to {destination} and hotel in {destination}.
                   you will focus on finding the cheapest flights and hotels for residence.
                   They should be properly arranged.
                   and provide links also."""),
    tools = [serper,scrape],
    agent = booking_agent,
    async_execution = True,
    expected_output = "blocks with in headings either trains,flights or hotels.",
    output_file = "availables.md" 
)

In [13]:
main_crew = Crew(
    agents = [booking_agent],
    tasks = [booking_task],
    process = Process.sequential,
)

2025-01-24 13:54:54,554 - 10664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [14]:
results = main_crew.kickoff(inputs={"start_trip":"Delhi","destination":"Bhopal","Budget" : 20000,"people":2,"day":2})



> Entering new CrewAgentExecutor chain...
Thought: I need to find the cheapest flights or trains from Delhi to Bhopal and hotels in Bhopal for 2 people with a 2-night stay, within a budget of ₹20,000.  I'll start by comparing flight and train prices.

Action: Search the internet

Action Input: {"search_query": "Delhi to Bhopal flights and trains"}
 


Search results: Title: Delhi To Bhopal Flights - MakeMyTrip
Link: https://www.makemytrip.com/flights/new_delhi-bhopal-cheap-airtickets.html
Snippet: The minimum airfare for a Delhi to Bhopal flights would be 3299, which may go up to 11325 depending on the route, booking time and availability. It is ...
---
Title: Cheap Flight Deals from Indira Gandhi Intl to Bhopal (DEL-BHO)
Link: https://www.kayak.com/flight-routes/New-Delhi-Indira-Gandhi-Intl-DEL/Bhopal-BHO
Snippet: Recent one-way flight deals from New Delhi to Bhopal ; 11:20 am - 12:35 pm · 1h 15m · $39 ; 7:05 am - 8:25 am · 1h 20m · $40 ; 5:55 am - 7:20 am · 1h 25m · $41 ; 5:55 am .

In [16]:
from IPython.display import Markdown, display

# Path to your Markdown file


# Open and read the Markdown file
with open("availables.md", "r", encoding="utf-8") as md_file:
    content = md_file.read()

# Render and display the Markdown content in the notebook
display(Markdown(content))


# Delhi to Bhopal Trip for 2 People (2 Nights) - Budget ₹20,000

This itinerary prioritizes the cheapest options while maintaining a reasonable level of comfort. Prices are estimates and may vary depending on the time of booking and availability.  Always double-check prices on the websites before booking.


## Flights

* **Estimated Cost:** ₹6598 (₹3299 per person, based on MakeMyTrip's lowest fare).
* **Airline:** IndiGo (or similar budget airline offering the lowest fare at the time of booking).
* **Booking Website:** MakeMyTrip ([https://www.makemytrip.com/flights/new_delhi-bhopal-cheap-airtickets.html](https://www.makemytrip.com/flights/new_delhi-bhopal-cheap-airtickets.html))  *Check Goibibo ([https://www.goibibo.com/flights/delhi-to-bhopal-flights/](https://www.goibibo.com/flights/delhi-to-bhopal-flights/)) for comparison.*

## Hotels

* **Estimated Cost:** ₹4000 (₹2000 per night for a double room, based on MakeMyTrip budget hotels).
* **Hotel:**  This requires further research on MakeMyTrip ([https://www.makemytrip.com/hotels/bhopal-hotels.html](https://www.makemytrip.com/hotels/bhopal-hotels.html))  to find a hotel under ₹2000 per night with good reviews.  Filter by price and user rating to find the best option within the budget.  Consider location relative to your activities in Bhopal.
* **Booking Website:** MakeMyTrip ([https://www.makemytrip.com/hotels/bhopal-hotels.html](https://www.makemytrip.com/hotels/bhopal-hotels.html)) *Check Goibibo ([https://www.goibibo.com/hotels/hotels-in-bhopal-ct/](https://www.goibibo.com/hotels/hotels-in-bhopal-ct/)) for comparison.*


**Note:**  The total estimated cost (₹6598 flights + ₹4000 hotel = ₹10598) is well within the ₹20,000 budget.  The remaining budget can be used for meals, local transportation, activities, or to upgrade to slightly more expensive flights or hotels if desired.  Remember to book flights and hotels as early as possible to secure the best deals.

In [17]:
support_agent = Agent(
    role="Customer Support",
    goal="Provide real-time assistance and resolve any travel-related queries.",
    backstory="A helpful assistant, ready to guide users and solve their problems immediately.",
    memory = True,
    allow_deligation = True,
    verbose = True,
    tools = [serper,scrape],
    llm = model)

In [20]:
support_task = Task(
    description = ("""Provide the assistance to the user for any {Query}."""),
    expected_output = "give a text output that will assistance for the {Query}",
    agent = support_agent,
    tool = [serper,scrape],
)

In [21]:
support_crew = Crew(
    agents = [support_agent],
    tasks = [support_task],
    process= Process.sequential,
)

2025-01-24 14:20:16,043 - 10664 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [24]:
support_1 = support_crew.kickoff(inputs={"Query":"provide me the youtube link for how to make a crewai tool."})



> Entering new CrewAgentExecutor chain...
Thought:The user is asking for a YouTube link on how to make a CrewAI tool.  I need to search YouTube for relevant videos.  Since I can't directly access and browse YouTube, I'll need to search the internet for links to relevant YouTube videos.

Action: Search the internet

Action Input: {"search_query": "youtube tutorial crewai tool creation"}
 


Search results: Title: Crew AI Full Tutorial For Beginners - Build Your Own AI Agents
Link: https://www.youtube.com/watch?v=q6QLGS306d0
Snippet: ... Tool 24:52 Test it! 27:45 3rd Project - Ollama Local AI 35:28 4th Project - Use Starter Example 37:30 CrewAI Tool - PDF RAG 47:45 Outro ...
---
Title: Let's Create a CUSTOM AI Tool with CrewAI | [Day 4 of 8] - YouTube
Link: https://www.youtube.com/watch?v=rcmMK-zkxrQ
Snippet: Let's Create a CUSTOM AI Tool with CrewAI | [Day 4 of 8]. 3.4K views · 3 months ago ...more. Tyler AI. 14.4K. Subscribe. 97. Share.
---
Title: CrewAI Tutorial: Complete Crash Cour

In [25]:
print(support_1)

I found several YouTube videos that may help you learn how to make a CrewAI tool.  Here are some links:

* https://www.youtube.com/watch?v=q6QLGS306d0 (Crew AI Full Tutorial For Beginners - Build Your Own AI Agents)
* https://www.youtube.com/watch?v=rcmMK-zkxrQ (Let's Create a CUSTOM AI Tool with CrewAI | [Day 4 of 8])
* https://www.youtube.com/watch?v=sPzc6hMg7So (CrewAI Tutorial: Complete Crash Course for Beginners)
* https://www.youtube.com/watch?v=kBXYFaZ0EN0 (CrewAI Step-by-Step | Complete Course for Beginners)
* https://www.youtube.com/watch?v=UV81LAb3x2g (crewAI Crash Course For Beginners-How To Create Multi AI Agent)
* https://www.youtube.com/watch?v=otHvxzQvq0s (Crew AI for Beginners - Easy Introduction)
* https://www.youtube.com/watch?v=2Uar1fDKYpQ (CrewAI - Building a Custom Crew)
* https://medium.com/@fatikir15/how-to-create-tools-for-your-ai-team-a-youtube-blog-post-generator-using-crewai-and-gemini-pro-1b65f13ff2ca (This is a Medium article, not YouTube, but it might be h